# RIOT API Spielerei

## 1. Extract

In [2]:
from riotwatcher import LolWatcher

#initialize user by SummonerName
lol_watcher = LolWatcher('RGAPI-53b44f03-ad6b-400a-839b-58b9b4de5c16')
my_region_old = "euw1"
my_region_new = "europe"
me = lol_watcher.summoner.by_name(my_region_old, 'Dödelmann')
me

{'id': 'MOl9TayWUqyJUqWjiWmZ_8LCJ2Mx7dIGrbYd2c-KczGSq94',
 'accountId': 'c5ISf-B6kPS8Dy6j-UJZePMCZvudyumcozcx-uzVZBmtNJw',
 'puuid': 'd_bQaUdFqci9l6ZVZZK19LMaZy5gFQzB-SNf_7LyWJTvzKOBy8wWhWCCtcEqRSoH9yma8JtHGtez5w',
 'name': 'Dödelmann',
 'profileIconId': 3552,
 'revisionDate': 1645130165000,
 'summonerLevel': 223}

In [3]:
#get gameIds of the user
matchlist_ids = []
while len(matchlist_ids)%10 == 0:
    matchlist_ids.extend(lol_watcher.match.matchlist_by_puuid(my_region_new, me['puuid'], count=100, start=len(matchlist_ids)))

#TODO: Doesn't find all historical games (matchv5 issue?)

In [6]:
#get matches by matchId and append to list (takes a while)
matchlist = []
#Riot API is limited to 100 requests/min
for match_id in matchlist_ids:
    #for i in tqdm(range(len(matchlist_ids))):
    matchlist.append(lol_watcher.match.by_id(my_region_new, match_id))
    print(match_id + "found and added")
    print("Progress:" + str(matchlist_ids.index(match_id)) + "/" + str(len(matchlist_ids)))

#TODO: Skip on 404

EUW1_5736207198found and added
Progress:0/957
EUW1_5736162941found and added
Progress:1/957
EUW1_5723314448found and added
Progress:2/957
EUW1_5723297599found and added
Progress:3/957
EUW1_5723292963found and added
Progress:4/957
EUW1_5722163932found and added
Progress:5/957
EUW1_5722020340found and added
Progress:6/957
EUW1_5722035652found and added
Progress:7/957
EUW1_5722032619found and added
Progress:8/957
EUW1_5721995587found and added
Progress:9/957
EUW1_5719837820found and added
Progress:10/957
EUW1_5719861803found and added
Progress:11/957
EUW1_5719807273found and added
Progress:12/957
EUW1_5716494527found and added
Progress:13/957
EUW1_5716437884found and added
Progress:14/957
EUW1_5713701607found and added
Progress:15/957
EUW1_5713647531found and added
Progress:16/957
EUW1_5713602655found and added
Progress:17/957
EUW1_5713526468found and added
Progress:18/957
EUW1_5710444717found and added
Progress:19/957
EUW1_5710348932found and added
Progress:20/957
EUW1_5708438033found an

HTTPError: 404 Client Error: Not Found for url: https://europe.api.riotgames.com/lol/match/v5/matches/EUW1_5322091733

In [7]:
#backup matchlist
import json
with open('C:\\Users\\schmi\\OneDrive\\StudiumS5\\DSBD\\Spark_Training/matchlist.json', 'w') as fout:
    json.dump(matchlist , fout)
matchlist_copy = matchlist

## 2. Transform

In [2]:
import pandas as pd
matches = [] #new list to flatten the data
for match_data in matchlist_copy:
    match = [match_data["metadata"]["matchId"]]
    match.append([player["win"] for player in match_data["info"]["participants"] if player["puuid"] == me["puuid"]][0]) #checks all participants and checks if user was winning
    match.append([player["championName"] for player in match_data["info"]["participants"] if player["puuid"] == me["puuid"]][0])
    for player in match_data["metadata"]["participants"]: #appends playerIds
        match.append(player)
    matches.append(match)


matches_df = pd.DataFrame(matches) #add flattened data to new Dataframe
matches_df.columns=['Match ID', 'win', 'Champ','Player1', 'Player2', 'Player3', 'Player4', 'Player5', 'Player6', 'Player7', 'Player8', 'Player9', 'Player10']
matches_df

NameError: name 'matchlist_copy' is not defined

## 3. Load